In [1]:
import pandas as pd 
import numpy as np

from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

In [2]:
import os
for dirname, _, filenames in os.walk('C:\Eric\M5_Data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Eric\M5_Data\calendar.csv
C:\Eric\M5_Data\calendar_add_snap.csv
C:\Eric\M5_Data\disaster_minji.csv
C:\Eric\M5_Data\sales_train_evaluation.csv
C:\Eric\M5_Data\sales_train_validation.csv
C:\Eric\M5_Data\sample_submission.csv
C:\Eric\M5_Data\sell_prices.csv
C:\Eric\M5_Data\SNAP_CA.csv
C:\Eric\M5_Data\SNAP_TX.csv
C:\Eric\M5_Data\SNAP_WI.csv


In [3]:
SNAP_pl_ca = pd.read_csv("C:\Eric\M5_Data\SNAP_CA.csv")
SNAP_pl_tx = pd.read_csv("C:\Eric\M5_Data\SNAP_TX.csv")
SNAP_pl_wi = pd.read_csv("C:\Eric\M5_Data\SNAP_WI.csv")

In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

#
# reduce_mem_usage(SNAP_pl_ca)
# reduce_mem_usage(SNAP_pl_tx)
# reduce_mem_usage(SNAP_pl_wi)


In [11]:
CAL_DTYPES={"event_name_1": "category", 
            "event_name_2": "category", 
            "event_type_1": "category", 
            "event_type_2": "category", 
            "weekday": "category", 
            'wm_yr_wk': 'int16', 
            "wday": "int16",
            "month": "int16", "year": "int16", 
            "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }

PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [37]:
def create_eric_dt(nrows= None):

    prices =  pd.read_csv("C:\Eric\M5_Data\sell_prices.csv")
    reduce_mem_usage(prices)
    
    cal = pd.read_csv("C:\Eric\M5_Data\calendar_add_snap.csv")
    reduce_mem_usage(cal)
    
    numcols = [f"d_{day}" for day in range(1,1913+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    
    dt = pd.read_csv("C:\Eric\M5_Data\sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    dt['total_sales'] = dt['sales'] * dt['sell_price']
    reduce_mem_usage(dt)
    
    return dt


In [38]:
%%time

eric_data = create_eric_dt(nrows=None)
eric_data.shape

Memory usage after optimization is: 130.48 MB
Decreased by 37.5%
Memory usage after optimization is: 0.14 MB
Decreased by 43.4%
Memory usage after optimization is: 5179.69 MB
Decreased by 3.3%
Wall time: 6min 22s


In [39]:
%%time

eric_data.to_csv("eric_train_validation.csv",index=False)

Wall time: 11min 35s


In [42]:
new_date = pd.read_csv("C:\\Users\Seoul IT\\eric_train_validation.csv")

C:\Users\Seoul IT\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
new_date.shape

(46027957, 26)

In [44]:
new_date.columns

Index(['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'ca_recipient', 'wi_recipient',
       'wx_recipient', 'sell_price', 'total_sales'],
      dtype='object')

In [47]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [52]:
%%time

da = create_fea(eric_data)
da.shape

AttributeError: Can only use .dt accessor with datetimelike values

In [53]:
reduce_mem_usage(da)

NameError: name 'da' is not defined

In [ ]:
da.info()

In [ ]:
da.dropna(inplace = True)
da.shape

In [ ]:
%%time

eric_data = create_eric_dt(nrows=None)
eric_data.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.dropna(inplace = True)
df.shape

In [ ]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

In [25]:
dt001 = pd.read_csv("C:\Eric\M5_Data\sales_train_validation.csv", )

In [27]:
dt001.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1919)

In [28]:
dt002 = pd.read_csv("C:\Eric\M5_Data\sell_prices.csv")

In [31]:
dt002.columns

Index(['store_id', 'item_id', 'wm_yr_wk', 'sell_price'], dtype='object')

In [35]:
del SNAP_pl_ca, SNAP_pl_tx, SNAP_pl_wi 

In [40]:
whos

Variable           Type         Data/Info
-----------------------------------------
CAL_DTYPES         dict         n=12
PRICE_DTYPES       dict         n=4
create_eric_dt     function     <function create_eric_dt at 0x000001FF812E09D8>
datetime           type         <class 'datetime.datetime'>
dirname            str          C:\Eric\M5_Data
eric_data          DataFrame                             <...>027957 rows x 26 columns]
filename           str          SNAP_WI.csv
filenames          list         n=10
gc                 module       <module 'gc' (built-in)>
lgb                module       <module 'lightgbm' from '<...>\\lightgbm\\__init__.py'>
np                 module       <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
os                 module       <module 'os' from 'C:\\Us<...>\\Anaconda3\\lib\\os.py'>
pd                 module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
prices             DataFrame            store_id        i<...>6841121 rows x 

In [23]:
%%time

eric_data = create_eric_dt(nrows=None)
eric_data.shape

Memory usage after optimization is: 130.48 MB
Decreased by 37.5%
Memory usage after optimization is: 0.14 MB
Decreased by 43.4%


KeyError: 'sell_prices'

In [ ]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [ ]:
%%time

create_fea(df)
df.shape